# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm_notebook

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []
# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10000)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    data = citipy.nearest_city(lat_lng[0], lat_lng[1])
    city = data.city_name
    country = data.country_code
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)
# Print the city count to confirm sufficient count
len(cities)

2235

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
city_data = {'City' : cities, 'Country' : countries}
city_data_df = pd.DataFrame(city_data)
city_data_df['Lat'] = ''
city_data_df['Lng'] = ''
city_data_df['Temperature (F)'] = ''
city_data_df['Humidity (%)'] = ''
city_data_df['Cloudiness (%)'] = ''
city_data_df['Wind Speed (mph)'] = ''
city_data_df.head(6)

,City,Country,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,rikitea,pf,,,,,,
1,carbondale,us,,,,,,
2,new norfolk,au,,,,,,
3,tuktoyaktuk,ca,,,,,,
4,esperance,au,,,,,,
5,hobart,au,,,,,,


In [4]:
#units = 'Imperial'
#url = "http://api.openweathermap.org/data/2.5/weather?"
#query_url = f"{url}appid={api_key}&units={units}&q=saleaula,ws"
#print(query_url)

In [ ]:
temp = []
units = 'Imperial'
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={api_key}&units={units}&q="
# Loop through the cities_pd and run a lat/long search for each city
for index, row in tqdm_notebook(list(city_data_df.iterrows())):
    
    city = row['City']
    country = row['Country']
    response = requests.get(query_url + city +',' + country).json()
    
    try:
        #print(f"Data is being retieved for {city}, {country}.")
        city_data_df.loc[index, 'Lng'] = response['coord']['lon']
        city_data_df.loc[index, 'Temperature (F)'] = response['main']['temp']
        city_data_df.loc[index, 'Humidity (%)'] = response['main']['humidity']
        city_data_df.loc[index, 'Lat'] = response['coord']['lat']
        city_data_df.loc[index, 'Cloudiness (%)'] = response['clouds']['all']
        city_data_df.loc[index, 'Wind Speed (mph)'] = response['wind']['speed']
    except (KeyError, IndexError):
        #print("Missing field/result... skipping.")
        city_data_df.loc[index, 'Temperature (F)'] = np.nan
        city_data_df.loc[index, 'Humidity (%)'] = np.nan
        city_data_df.loc[index, 'Lat'] = np.nan
        city_data_df.loc[index, 'Lng'] = np.nan
        city_data_df.loc[index, 'Cloudiness (%)'] = np.nan
        city_data_df.loc[index, 'Wind Speed (mph)'] = np.nan

In [ ]:
#replace humidity reading over 100% with NaN
for _, row in city_data_df.iterrows():
    if row['Humidity (%)'] > 100:
        row['Humidity (%)'] = np.nan

In [ ]:
# Drop Rows with NaN Values
city_data_df = city_data_df.dropna()

In [ ]:
# Visualize Data
city_data_df.describe()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#Export to CSV file
city_data_df.to_csv('City_Weather.csv', index=False)

In [ ]:
city_data_df.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
city_data_df['color'] = ''
for _, row in city_data_df.iterrows():
    if row['Temperature (F)'] < 40:
        row['color'] = 'darkblue'
    if row['Temperature (F)'] > 40:
        row['color'] = 'dodgerblue'
    if row['Temperature (F)'] > 60:
        row['color'] = 'skyblue'
    if row['Temperature (F)'] > 70:
        row['color'] = 'gold'
    if row['Temperature (F)'] > 80:
        row['color'] = 'orange'
    if row['Temperature (F)'] > 90:
        row['color'] = 'red'

In [ ]:
# Added due to error with date response from server
try:
    date = pd.to_datetime(response['dt'], unit = 's').strftime('%m-%d-%Y')

except (KeyError, IndexError, ValueError):
    date = 'Date not available'
        

In [ ]:
# List of values x_axis values
x_axis = city_data_df['Lat']
data = city_data_df['Temperature (F)']
color = city_data_df['color']

#Plot scatter
plt.scatter(x_axis, data, marker="o", facecolors= color , edgecolors="black", alpha=0.70)

#Format plot
plt.grid()
plt.xlabel('Latitude', size='12')
plt.ylabel('Temperature (F)', size='12')
plt.title(f"City Latitude vs Temperature ({date})", size='14')
plt.tight_layout()
plt.savefig('LatvTemp')
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
# List of values x_axis values
x_axis = city_data_df['Lat']
data = city_data_df['Humidity (%)']
#color = city_data_df['color']

#Plot scatter
plt.scatter(x_axis, data, marker="o", facecolors= 'dodgerblue' , edgecolors="black", alpha=0.70)

#Format plot
plt.grid()
plt.xlabel('Latitude', size='12')
plt.ylabel('Humidity (%)', size='12')
plt.title(f"City Latitude vs Humidity ({date})", size='14')
plt.tight_layout()
plt.savefig('LatvHum')
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
# List of values x_axis values
x_axis = city_data_df['Lat']
data = city_data_df['Cloudiness (%)']
#color = city_data_df['color']

#Plot scatter
plt.scatter(x_axis, data, marker="o", facecolors= 'dodgerblue' , edgecolors="black", alpha=0.70)

#Format plot
plt.grid()
plt.xlabel('Latitude', size='12')
plt.ylabel('Cloudiness (%)', size='12')
plt.title(f"City Latitude vs Cloudiness ({date})", size='14')
plt.tight_layout()
plt.savefig('LatvCloud')
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# List of values x_axis values
x_axis = city_data_df['Lat']
data = city_data_df['Wind Speed (mph)']
#color = city_data_df['color']

#Plot scatter
plt.scatter(x_axis, data, marker="o", facecolors= 'dodgerblue' , edgecolors="black", alpha=0.70)

#Format plot
plt.grid()
plt.xlabel('Latitude', size='12')
plt.ylabel('Wind Speed (mph)', size='12')
plt.title(f"City Latitude vs Wind Speed ({date})", size='14')
plt.tight_layout()
plt.savefig('LatvWind')
plt.show()